In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/DMNC/mimic_data

/content/drive/MyDrive/DMNC/mimic_data


In [3]:
!apt-get install python3.6

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3.6 is already the newest version (3.6.9-1~18.04ubuntu1.7).
python3.6 set to manually installed.
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.


In [5]:
%cd ../
!python3.6 get-pip.py "pip<22"

/content/drive/MyDrive/DMNC
     |████████████████████████████████| 1.7 MB 5.0 MB/s            
     |████████████████████████████████| 952 kB 72.8 MB/s            


In [6]:
!python3.6 -m pip install -r pip_requirements.txt

     |████████████████████████████████| 129 kB 5.1 MB/s            
     |████████████████████████████████| 2.8 MB 53.1 MB/s            
     |████████████████████████████████| 53 kB 2.0 MB/s             
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 156 kB 29.8 MB/s            
     |████████████████████████████████| 57 kB 4.4 MB/s             
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 97 kB 5.1 MB/s             
     |████████████████████████████████| 86 kB 4.1 MB/s             
     |████████████████████████████████| 93 kB 1.9 MB/s             
     |████████████████████████████████| 164 kB 51.7 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 94 kB 1.6 MB/s             
     |████████████████████████████████| 183 kB 3.6 MB/s            
     |████████████████████████████████| 829 kB 47.2 MB/s            
  Preparing metadata (setup.py) ... done
     |██████████

In [7]:
%cd ./mimic_data

/content/drive/MyDrive/DMNC/mimic_data


In [8]:
import pandas as pd

dict_patient = {}

df = pd.read_csv('./physionet.org/files/mimiciii/1.4/PATIENTS.csv')
patients = df['SUBJECT_ID'].unique()
dict_patient['pa'] = list(patients)
patients[0:5]

array([249, 250, 251, 252, 253])

In [9]:
df = pd.read_csv('./physionet.org/files/mimiciii/1.4/DIAGNOSES_ICD.csv')
diagnoses = df.groupby('SUBJECT_ID')['ICD9_CODE'].apply(list)

dict_patient['diag'] = []
for patient in patients:
  if patient not in diagnoses:
    dict_patient['diag'].append([])
  else:
    dict_patient['diag'].append(diagnoses[patient])

In [10]:
df = pd.read_csv('./physionet.org/files/mimiciii/1.4/PROCEDURES_ICD.csv')
procedures = df.groupby('SUBJECT_ID')['ICD9_CODE'].apply(list)

dict_patient['proc'] = []
for patient in patients:
  if patient not in procedures:
    dict_patient['proc'].append([])
  else:
    dict_patient['proc'].append(procedures[patient])

In [11]:
df = pd.read_csv('./physionet.org/files/mimiciii/1.4/PRESCRIPTIONS.csv')
prescriptions = df.groupby('SUBJECT_ID')['FORMULARY_DRUG_CD'].apply(list)

dict_patient['med'] = []
for patient in patients:
  if patient not in prescriptions:
    dict_patient['med'].append([])
  else:
    dict_patient['med'].append(prescriptions[patient])

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [12]:
df = pd.read_csv('../mimic_data/physionet.org/files/mimiciii/1.4/PRESCRIPTIONS.csv')
prescriptions = df.groupby('HADM_ID')['FORMULARY_DRUG_CD'].apply(list)

df = pd.read_csv('../mimic_data/physionet.org/files/mimiciii/1.4/ADMISSIONS.csv', usecols = ['HADM_ID', 'SUBJECT_ID', 'ADMITTIME'])
dict_patient = {}
dict_patient['pa'] = []
dict_patient['admit_time'] = []
dict_patient['med'] = []
dict_patient['proc'] = []
dict_patient['diag'] = []
for index, row in df.iterrows():
  dict_patient['pa'].append(row['SUBJECT_ID'])
  dict_patient['admit_time'].append(row['ADMITTIME'])
  if row['SUBJECT_ID'] in diagnoses:
    dict_patient['diag'].append(diagnoses[row['SUBJECT_ID']])
  else:
    dict_patient['diag'].append([])
  if row['HADM_ID'] in prescriptions:
    dict_patient['med'].append(prescriptions[row['HADM_ID']])
  else:
    dict_patient['med'].append([])
  if row['SUBJECT_ID'] in procedures:
    dict_patient['proc'].append(procedures[row['SUBJECT_ID']])
  else:
    dict_patient['proc'].append([])

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [14]:
%cd ../dual_task

from preprocess2_mimic import preprocess

# it only works for patients with multiple admit times
preprocess(dict_patient)

/content/drive/MyDrive/DMNC/dual_task
22
[[['9678', '9693', 'E9502', 'E9503', '3488', '29620', '4019'], ['PAD'], [9604, 9671, 9634], '2196-04-09 12:26:00']]
23
[[['2252', '3485', '78039', '4241', '4019', '2720', '2724', 'V4581', 'V4579', 'V1582', '41401', '4111', 'V4582', '60000', '3899'], ['SYR60ML', 'PHEN12SYR', 'NITR20SYR', 'LR1000', 'CARA1', 'LIDOBASE', 'LIDO100S', 'NS100', 'LIDE4500I', 'CEFA10I', 'KCLBASE2', 'GLYC1I', 'NEOSI', 'CALG1I', 'MAGS1I', 'MEPE50I', 'MORP0.5-4', 'METO10I', 'PERC', 'ACET325', 'ACET650R', 'ASA81EC', 'ASA81', 'RANI150', 'RANI150L', 'DOCU100', 'DOCU100L', 'NACLFLUSH', 'D5W250', 'NTGBASE', 'SOLN3', 'SOLN2', 'INHRIV', 'KCL20PM', 'PHEN10I', 'NITP50I', 'NTG100PB', 'PROP100IG', 'PROP500IG', 'METO25', 'FURO40I', 'FURO20', 'MICROK10', 'TERAZ5', 'INSULIN', 'BISA10R', 'MOM30L', 'MORP1-5', 'ATOR20', 'FERRG', 'KCL20P', 'METO50'], [3613, 3615, 3961, 8872, 9904, 9905, 9907, 151], '2153-09-03 07:15:00'], [['2252', '3485', '78039', '4241', '4019', '2720', '2724', 'V4581', 'V

In [15]:
!python3.6 -m pip install sklearn

  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 22.2 MB 1.5 MB/s            
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=488d86674401b3025c4c1eba5174749462bb8a4838b5ef7a57b4beb136f53fa9
  Stored in directory: /root/.cache/pip/wheels/23/9d/42/5ec745cbbb17517000a53cecc49d6a865450d1f5cb16dc8a9c
Successfully built sklearn


In [ ]:
!python3.6 emr_run.py --mode=train --use_mem=False --attend=128

Namespace(attend=128, dual_emb=True, emb_size=64, eval_step=3000, from_checkpoint='', hidden_dim=64, iters=42000, mem_size=16, mode='train', persist=True, read_heads=1, share_mem=True, task='trim_diag_proc_drug_no_adm', type='no_cache', use_beam_search=0, use_mem=False, use_teacher=False, word_size=64)
/content/drive/MyDrive/DMNC/dual_task/data/save/
Loading Data ... Done!
num_patient 1000
num train 666
num valid 167
num test 167
dim in  3172 1760
dim out 3172
2022-05-09 00:14:09.486741: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-05-09 00:14:09.505813: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2199995000 Hz
2022-05-09 00:14:09.506328: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x2c82840 executing computations on platform Host. Devices:
2022-05-09 00:14:09.506368: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor d